# **Plant Disease Detector**

**Gambaran Umum Proses**

![](https://images-provider.frontiersin.org/api/ipx/w=410&f=webp/https://www.frontiersin.org/files/Articles/1356260/fpls-15-1356260-HTML/image_m/fpls-15-1356260-g001.jpg)

# FETCH DATASET

In [1]:
!kaggle datasets download -d emmarex/plantdisease -p ./plantvillage

Dataset URL: https://www.kaggle.com/datasets/emmarex/plantdisease
License(s): unknown
100%|███████████████████████████████████████▉| 657M/658M [01:51<00:00, 7.37MB/s]
100%|████████████████████████████████████████| 658M/658M [01:51<00:00, 6.19MB/s]


# INITIAL

In [ ]:
import zipfile
import os
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# UNZIP FILE

In [ ]:
# Unzip the dataset file
zip_path = './plantvillage/plantdisease.zip'
extract_to_path = './plantvillage'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

print("Dataset unzipped successfully!")


# PROGRAM AND MODELS

In [ ]:
# Organize the dataset
# dataset_dir = os.path.join(extract_to_path, 'PlantVillage')

dataset_dir = os.path.join(extract_to_path, 'PlantVillage')

# Create directories for training and testing data
train_dir = os.path.join(extract_to_path, 'train')
test_dir = os.path.join(extract_to_path, 'test')

if not os.path.exists(train_dir):
    os.makedirs(train_dir)
if not os.path.exists(test_dir):
    os.makedirs(test_dir)

# Get class labels from subdirectories
class_names = os.listdir(dataset_dir)

for class_name in class_names:
    class_path = os.path.join(dataset_dir, class_name)
    if os.path.isdir(class_path):
        # Create directories for each class in train and test
        train_class_dir = os.path.join(train_dir, class_name)
        test_class_dir = os.path.join(test_dir, class_name)
        if not os.path.exists(train_class_dir):
            os.makedirs(train_class_dir)
        if not os.path.exists(test_class_dir):
            os.makedirs(test_class_dir)

        # List all images in the current class
        all_images = os.listdir(class_path)
        train_images, test_images = train_test_split(all_images, test_size=0.2, random_state=42)
        
        for image in train_images:
            shutil.copy(os.path.join(class_path, image), os.path.join(train_class_dir, image))
        for image in test_images:
            shutil.copy(os.path.join(class_path, image), os.path.join(test_class_dir, image))

print("Data organized into training and testing directories!")

# Image data generators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

# Build the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f'Test accuracy: {test_acc:.2f}')
print(f'Test loss: {test_loss:.2f}')

# Plot training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

# DF

In [ ]:
# Prepare dataset summary DataFrame
## Peview sinkat untuk gamabaran
import pandas as pd

def get_image_paths_and_labels(directory):
    image_paths = []
    labels = []
    for class_name in os.listdir(directory):
        class_path = os.path.join(directory, class_name)
        if os.path.isdir(class_path):
            for image_name in os.listdir(class_path):
                image_paths.append(os.path.join(class_path, image_name))
                labels.append(class_name)
    return image_paths, labels

train_image_paths, train_labels = get_image_paths_and_labels(train_dir)

# Create DataFrame
df = pd.DataFrame({
    'Image Path': train_image_paths,
    'Label': train_labels
})

# Display the DataFrame
print(df[['Image Path', 'Label']].head())

# Optional: Save the DataFrame to a CSV file
df.to_csv('dataset_summary.csv', index=False)

# Save Models 

In [ ]:
# Define the path to save the model
model_save_path = './plantvillage/model.h5'

# Save the entire model to the specified directory
model.save(model_save_path)

print(f"Model saved to {model_save_path}")

from tensorflow.keras.models import load_model

# Load the model from the specified directory
model = load_model(model_save_path)

print("Model loaded successfully!")


# UI and Deployment

In [ ]:
# Function to predict the class of a new image
def predict_image(image_path, model):
    img = load_img(image_path, target_size=(150, 150))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions[0])
    class_labels = list(train_generator.class_indices.keys())
    return class_labels[predicted_class]

# Example usage for prediction
new_image_path = 'path_to_new_image.jpg'  # Replace with the path to the new image
predicted_class = predict_image(new_image_path, model)
print(f'The predicted class for the image is: {predicted_class}')